In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os
from IPython.display import IFrame, display
import glob

yearly_folder = r'D:\yourfilename'
all_files = sorted(glob.glob(os.path.join(yearly_folder, '*.csv')))
df_list = []
for file in all_files:
    df_temp = pd.read_csv(file)
    df_temp['time'] = pd.to_datetime(df_temp['time'], utc=True)
    df_temp['time'] = df_temp['time'].dt.tz_convert('America/New_York')
    df_list.append(df_temp)
df = pd.concat(df_list, ignore_index=True)
df = df.sort_values('time').reset_index(drop=True)

def build_sessions(df):
    all_dates = pd.date_range(df['time'].dt.date.min(), df['time'].dt.date.max(), freq='D', tz='America/New_York')
    sessions = []
    for d in all_dates:
        weekday = d.weekday()
        if weekday == 0:
            session_start = pd.Timestamp.combine(d - pd.Timedelta(days=1), pd.to_datetime('18:00:00').time()).tz_localize('America/New_York')
            session_end = pd.Timestamp.combine(d, pd.to_datetime('16:59:00').time()).tz_localize('America/New_York')
            sessions.append({'session_date': d.date(), 'start': session_start, 'end': session_end})
        elif weekday in [1, 2, 3, 4]:
            session_start = pd.Timestamp.combine(d - pd.Timedelta(days=1), pd.to_datetime('18:00:00').time()).tz_localize('America/New_York')
            session_end = pd.Timestamp.combine(d, pd.to_datetime('16:59:00').time()).tz_localize('America/New_York')
            sessions.append({'session_date': d.date(), 'start': session_start, 'end': session_end})
    return pd.DataFrame(sessions)

sessions_df = build_sessions(df)

df = df.sort_values('time')
sessions_df = sessions_df.sort_values('start')
df = pd.merge_asof(
    df,
    sessions_df[['session_date', 'start', 'end']],
    left_on='time',
    right_on='start',
    direction='backward',
    tolerance=pd.Timedelta('24h')
)
df = df[(df['time'] >= df['start']) & (df['time'] <= df['end'])]
df['futures_session'] = df['session_date']

pio.renderers.default = 'iframe'

save_dir = r'D:\yourfilename'
os.makedirs(save_dir, exist_ok=True)

unique_sessions = sessions_df['session_date'].tolist()

pd_levels = []
fpfvg_records = []
for session_date in unique_sessions:
    session_row = sessions_df[sessions_df['session_date'] == session_date].iloc[0]
    session_start_dt = session_row['start']
    session_end_dt = session_row['end']
    session_df = df[(df['time'] >= session_start_dt) & (df['time'] <= session_end_dt)]
    pd_levels.append({
        'session_date': session_date,
        'high': session_df['high'].max() if not session_df.empty else np.nan,
        'low': session_df['low'].min() if not session_df.empty else np.nan
    })
pd_levels_df = pd.DataFrame(pd_levels)

for idx, session_date in enumerate(unique_sessions):
    session_row = sessions_df[sessions_df['session_date'] == session_date].iloc[0]
    session_start_dt = session_row['start']
    session_end_dt = session_row['end']
    session_df = df[(df['time'] >= session_start_dt) & (df['time'] <= session_end_dt)]

    shapes = [
        dict(
            type="rect",
            xref="paper",
            yref="paper",
            x0=0,
            y0=0,
            x1=1,
            y1=1,
            line=dict(
                color="Black",
                width=1
            )
        )
    ]

    annotations = []
    
    midnight = pd.Timestamp.combine(session_date, pd.to_datetime('00:00:00').time()).tz_localize('America/New_York')
    midnight_bar = session_df[session_df['time'] == midnight]
    if not midnight_bar.empty:
        midnight_open = midnight_bar['open'].iloc[0]
        shapes.append(dict(
            type="line",
            xref="x",
            yref="y",
            x0=midnight,
            y0=midnight_open,
            x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
            y1=midnight_open,
            line=dict(color="#0000ff", width=1, dash="solid"),
            layer="above"
        ))
        
        session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
        annotations.append(dict(
            x=session_9am,
            y=midnight_open,
            xref="x",
            yref="y",
            text="00:00 Open",
            showarrow=False,
            font=dict(color="#0000ff", size=10),
            xanchor="left",
            yanchor="bottom"
        ))

    pd_bsl = pd_ssl = None
    if idx > 0:
        prev_session = unique_sessions[idx - 1]
        prev_row = pd_levels_df[pd_levels_df['session_date'] == prev_session].iloc[0]
        pd_bsl = prev_row['high']
        pd_ssl = prev_row['low']
        pd_bsl_breached = (not session_df.empty) and (session_df['high'] >= pd_bsl).any()
        pd_ssl_breached = (not session_df.empty) and (session_df['low'] <= pd_ssl).any()

        if not pd.isna(pd_bsl):
            shapes.append(dict(
                type="line",
                xref="x",
                yref="y",
                x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
                y0=pd_bsl,
                x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
                y1=pd_bsl,
                line=dict(color="#801922", width=1, dash="solid"),
                layer="above"
            ))
            session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
            annotations.append(dict(
                x=session_9am,
                y=pd_bsl,
                xref="x",
                yref="y",
                text="PD BSL",
                showarrow=False,
                font=dict(color="#801922", size=10),
                xanchor="left",
                yanchor="bottom"
            ))
        
        if not pd.isna(pd_ssl):
            shapes.append(dict(
                type="line",
                xref="x",
                yref="y",
                x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
                y0=pd_ssl,
                x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
                y1=pd_ssl,
                line=dict(color="#801922", width=1, dash="solid"),
                layer="above"
            ))
            session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
            annotations.append(dict(
                x=session_9am,
                y=pd_ssl,
                xref="x",
                yref="y",
                text="PD SSL",
                showarrow=False,
                font=dict(color="#801922", size=10),
                xanchor="left",
                yanchor="top"
            ))
    
    open_930 = pd.Timestamp.combine(session_date, pd.to_datetime('09:30:00').time()).tz_localize('America/New_York')
    session_df_after_930 = session_df[session_df['time'] >= open_930]

    fvg_list = []
    for i in range(1, len(session_df_after_930) - 1):
        candle_1 = session_df_after_930.iloc[i - 1]
        candle_2 = session_df_after_930.iloc[i]
        candle_3 = session_df_after_930.iloc[i + 1]
        if candle_3['low'] > candle_1['high']:
            fvg_list.append({
                'orientation': 'BISI',
                'start_time': candle_1['time'],
                'end_time': session_end_dt,
                'high': candle_1['high'],
                'low': candle_3['low'],
                'idx': i
            })
        if candle_3['high'] < candle_1['low']:
            fvg_list.append({
                'orientation': 'SIBI',
                'start_time': candle_1['time'],
                'end_time': session_end_dt,
                'high': candle_3['high'],
                'low': candle_1['low'],
                'idx': i
            })

    if fvg_list:
        first_fvg = fvg_list[0]
        shapes.append(dict(
            type="rect",
            xref="x",
            yref="y",
            x0=first_fvg['start_time'],
            y0=first_fvg['high'] if first_fvg['orientation'] == 'BISI' else first_fvg['high'],
            x1=first_fvg['end_time'],
            y1=first_fvg['low'] if first_fvg['orientation'] == 'BISI' else first_fvg['low'],
            fillcolor="rgba(0, 255, 0, 0.2)" if first_fvg['orientation'] == 'BISI' else "rgba(255, 0, 0, 0.2)",
            line=dict(width=0),
            layer="below"
        ))
        annotations.append(dict(
            x=session_9am,
            y=first_fvg['low'] if first_fvg['orientation'] == 'BISI' else first_fvg['low'],
            xref="x",
            yref="y",
            text="F.P. " + first_fvg['orientation'],
            showarrow=False,
            font=dict(color="green" if first_fvg['orientation'] == 'BISI' else "red", size=10),
            xanchor="left",
            yanchor="middle"
        ))
        fpfvg_records.append({
            'session_date': session_date,
            'orientation': first_fvg['orientation'],
            'start_time': first_fvg['start_time'],
            'end_time': first_fvg['end_time'],
            'high': first_fvg['high'],
            'low': first_fvg['low']
        })

        for fvg in fvg_list[1:]:
            if fvg['orientation'] != first_fvg['orientation']:
                shapes.append(dict(
                    type="rect",
                    xref="x",
                    yref="y",
                    x0=fvg['start_time'],
                    y0=fvg['high'] if fvg['orientation'] == 'BISI' else fvg['high'],
                    x1=fvg['end_time'],
                    y1=fvg['low'] if fvg['orientation'] == 'BISI' else fvg['low'],
                    fillcolor="rgba(0, 255, 0, 0.2)" if fvg['orientation'] == 'BISI' else "rgba(255, 0, 0, 0.2)",
                    line=dict(width=0),
                    layer="below"
                ))
                annotations.append(dict(
                    x=session_9am,
                    y=fvg['low'] if fvg['orientation'] == 'BISI' else fvg['low'],
                    xref="x",
                    yref="y",
                    text="F.P. " + fvg['orientation'],
                    showarrow=False,
                    font=dict(color="green" if fvg['orientation'] == 'BISI' else "red", size=10),
                    xanchor="left",
                    yanchor="middle"
                ))
                break

    session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('09:00:00').time()).tz_localize('America/New_York')
    session_noon = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
    zoom_df = session_df[(session_df['time'] >= session_9am) & (session_df['time'] <= session_noon)]
    zoom_high = zoom_df['high'].max() if not zoom_df.empty else np.nan
    zoom_low = zoom_df['low'].min() if not zoom_df.empty else np.nan
    y_buffer = (zoom_high - zoom_low) * 0.1 if not np.isnan(zoom_high) and not np.isnan(zoom_low) else 1
    
    fig = go.Figure()
    if not session_df.empty:
        fig.add_trace(
            go.Candlestick(
                x=session_df['time'],
                open=session_df['open'],
                high=session_df['high'],
                low=session_df['low'],
                close=session_df['close'],
                name=str(session_date),
                increasing_line_color='#000000',
                increasing_fillcolor='#66bb6a',
                increasing_line_width=1,
                decreasing_line_color='#000000',
                decreasing_fillcolor='#000000',
                decreasing_line_width=1
            )
        )
        
    fig.update_layout(
        title=f'{session_date}',
        xaxis_title='time',
        yaxis_title='Price',
        autosize=True,
        height=600,
        showlegend=False,
        plot_bgcolor='#d0d4de',
        paper_bgcolor='#d0d4de',
        shapes=shapes,
        annotations=annotations,
        yaxis=dict(
            range=[zoom_low - y_buffer, zoom_high + y_buffer] if not np.isnan(zoom_high) and not np.isnan(zoom_low) else None
        ),
         xaxis=dict(
            range=[session_9am, session_noon],
            showgrid=False,
            rangeslider=dict(visible=False)
         ),
    )

    fig.update_xaxes(showgrid=False, rangeslider=dict(visible=False))
    fig.update_yaxes(showgrid=False, tickformat=',.2f', fixedrange=False)

    file_path = os.path.join(save_dir, f'fpfvg_{session_date}.html')
    fig.write_html(file_path)
    display(IFrame(src=file_path, width='100%', height=700))

fpfvg_df = pd.DataFrame(fpfvg_records)